[Link from the first notebook](./01_scraping_notebook_v2.ipynb)

# Explanatory Data Analysis

## A. EDA Set-up

Since this is a new notebook, there's a need to import certain scripts.

In [1]:
# Import basic libraries
import pandas as pd
import numpy as np

In [2]:
#!pip install matplotlib-venn (uncomment if the module isn't imported)

# Import visualisation libraries
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns
import pandas as pd
import numpy as np
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import ast
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from collections import Counter
import os
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

Importing the extracted data into this notebook.

In [3]:
ten_tropes_df = pd.read_csv('10_tropes.csv')
wiki_df  = pd.read_csv('simpsons_episode_data2.csv')

Adjusting the output view.

In [4]:
# some display adjustments to account for the fact that we have many columns
# and some columns contain many characters

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 800)

In [5]:
#checking to see what each header's type is for both dataframes

print(ten_tropes_df.columns)
print(wiki_df.columns)

Index(['Trope Name', 'Trope Description', 'Text Length', 'text_lemma'], dtype='object')
Index(['Episode Title', 'Full Story', 'Tropes'], dtype='object')


In [6]:
#Seeing what makes up the ten tropes dataframe
ten_tropes_df.describe()
ten_tropes_summary_stats = ten_tropes_df.describe()
print(ten_tropes_summary_stats)

       Text Length
count     10.00000
mean    1820.70000
std     1361.34101
min      642.00000
25%      877.25000
50%     1570.00000
75%     2150.25000
max     5204.00000


In [7]:
#Doing the same for the wiki dataframe
wiki_df.describe()
wiki_summary_stats = wiki_df.describe()
print(wiki_summary_stats)

                            Episode Title  \
count                                  13   
unique                                 13   
top     Simpsons Roasting on an Open Fire   
freq                                    1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [8]:
# Check data types of each column for the ten tropes dataframe
ten_tropes_data_types = ten_tropes_df.dtypes
print(ten_tropes_data_types)

# Check for missing values in each column
ten_tropes_df_missing_values = ten_tropes_df.isnull().sum()
print(ten_tropes_df_missing_values)

Trope Name           object
Trope Description    object
Text Length           int64
text_lemma           object
dtype: object
Trope Name           0
Trope Description    0
Text Length          0
text_lemma           0
dtype: int64


In [9]:
# Doing the same for wiki_df
wiki_data_types = wiki_df.dtypes
print(wiki_data_types)

# Check for missing values in each column
wiki_missing_values = wiki_df.isnull().sum()
print(wiki_missing_values)

Episode Title    object
Full Story       object
Tropes           object
dtype: object
Episode Title    0
Full Story       0
Tropes           0
dtype: int64


In [10]:
# Iterate through each column in ten_tropes_df and count unique values (for list-type columns) first
for column in ten_tropes_df.columns:
    if isinstance(ten_tropes_df[column].iloc[0], list):
        unique_values = []
        for sublist in ten_tropes_df[column]:
            for val in sublist:
                if val not in unique_values:
                    unique_values.append(val)
        unique_values_count = len(unique_values)
        print(f"Number of unique values in '{column}': {unique_values_count}")
    else:
        print(f"Skipped column '{column}' as it is not a list-type column.")

Skipped column 'Trope Name' as it is not a list-type column.
Skipped column 'Trope Description' as it is not a list-type column.
Skipped column 'Text Length' as it is not a list-type column.
Skipped column 'text_lemma' as it is not a list-type column.


In [11]:
# Done the same for wiki_df
for column in wiki_df.columns:
    if isinstance(wiki_df[column].iloc[0], list):
        unique_values = []
        for sublist in wiki_df[column]:
            for val in sublist:
                if val not in unique_values:
                    unique_values.append(val)
        unique_values_count = len(unique_values)
        print(f"Number of unique values in '{column}': {unique_values_count}")
    else:
        print(f"Skipped column '{column}' as it is not a list-type column.")

Skipped column 'Episode Title' as it is not a list-type column.
Skipped column 'Full Story' as it is not a list-type column.
Skipped column 'Tropes' as it is not a list-type column.


Firstly, we need to see how many tropes does each episode have. (Note to self, from the 552 tropes, select ten who appear the most in the simpsons episode related feature)

In [13]:
ten_tropes_df

Trope Name  \
0                   Imagine Spot   
1                      Couch Gag   
2                    Catchphrase   
3    Comically Missing the Point   
4                    Running Gag   
5   Disproportionate Retribution   
6  Establishing Character Moment   
7                  Karma Houdini   
8    Characterization Marches On   
9                  Special Guest   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [14]:
# Get unique trope names from the DataFrame
trope_names = ten_tropes_df["Trope Name"].unique()
trope_names

array(['Imagine Spot', 'Couch Gag', 'Catchphrase',
       'Comically Missing the Point', 'Running Gag',
       'Disproportionate Retribution', 'Establishing Character Moment',
       'Karma Houdini', 'Characterization Marches On', 'Special Guest'],
      dtype=object)

In [15]:
# Get the NLTK stopwords
nltk_stopwords = set(stopwords.words('english'))

# Get unique trope names from the DataFrame
trope_names = ten_tropes_df["Trope Name"].unique()

# Create a directory to save the bar chart images
output_directory = "bar_charts"
os.makedirs(output_directory, exist_ok=True)

# Loop through each trope and perform EDA
custom_stopwords = {
    'Imagine Spot': ['stopword1', 'stopword2', 'additional_stopword_for_Imagine_Spot'],
    'Couch Gag': ['stopword3', 'stopword4', 'additional_stopword_for_Couch_Gag'],
    'Catchphrase': ['stopword5', 'stopword6', 'additional_stopword_for_Catchphrase'],
    'Comically Missing the Point': ['stopword7', 'stopword8', 'additional_stopword_for_Comically_Missing_the_Point'],
    'Running Gag': ['stopword9', 'stopword10', 'additional_stopword_for_Running_Gag'],
    'Disproportionate Retribution': ['stopword11', 'stopword12', 'additional_stopword_for_Disproportionate_Retribution'],
    'Establishing Character Moment': ['stopword13', 'stopword14', 'additional_stopword_for_Establishing_Character_Moment'],
    'Karma Houdini': ['stopword15', 'stopword16', 'additional_stopword_for_Karma_Houdini'],
    'Characterization Marches On': ['stopword17', 'stopword18', 'additional_stopword_for_Characterization_Marches_On'],
    'Special Guest': ['stopword19', 'stopword20', 'additional_stopword_for_Special_Guest'],
}

# Loop through each trope and perform EDA
for trope_name in trope_names:
    # Filter the data for the current trope
    trope_data = ten_tropes_df[ten_tropes_df["Trope Name"] == trope_name]

    # Tokenize the lemmatized descriptions and remove common stopwords and custom stopwords
    all_words = " ".join(trope_data["text_lemma"])
    all_words = word_tokenize(all_words)
    
    # Remove common stopwords
    filtered_words = [word for word in all_words if word.lower() not in nltk_stopwords]
    
    # Remove custom stopwords
    filtered_words = [word for word in filtered_words if word.lower() not in custom_stopwords.get(trope_name, [])]

    # Count word frequencies
    word_freq = Counter(filtered_words)

    # Visualize the top occurring words
    top_words = word_freq.most_common(10)
    word, frequency = zip(*top_words)

    # Create a bar plot
    plt.figure(figsize=(10, 6))
    plt.bar(word, frequency)
    plt.title(f"Top Words in {trope_name}")
    plt.xlabel("Words")
    plt.ylabel("Frequency")
    plt.xticks(rotation=45)

    # Save the bar chart to a file in the output directory
    image_file_path = os.path.join(output_directory, f"{trope_name}_bar_chart.png")
    plt.savefig(image_file_path)
    plt.close()  # Close the plot to release resources

# All bar charts have been saved to the output_directory

Since apparently Establishing Character Moment and Karma Houdini both have the word Character appear the most, singular words aren't advised as the model will be unable to tell which is which. Instead, we'll use bigrams.

In [16]:
# Create a directory to save the bar chart images
output_directory = "bar_charts_bigrams"
os.makedirs(output_directory, exist_ok=True)

# Loop through each trope and perform EDA on bigrams
for trope_name in trope_names:
    # Filter the data for the current trope
    trope_data = ten_tropes_df[ten_tropes_df["Trope Name"] == trope_name]

    # Tokenize the lemmatized descriptions and remove common stopwords and custom stopwords
    all_words = " ".join(trope_data["text_lemma"])
    all_words = word_tokenize(all_words)
    
    # Remove common stopwords
    filtered_words = [word for word in all_words if word.lower() not in nltk_stopwords]
    
    # Remove custom stopwords
    filtered_words = [word for word in filtered_words if word.lower() not in custom_stopwords.get(trope_name, [])]

    # Create bigrams
    bigrams = list(ngrams(filtered_words, 2))

    # Count bigram frequencies
    bigram_freq = Counter(bigrams)

    # Visualize the top occurring bigrams
    top_bigrams = bigram_freq.most_common(10)
    bigram, frequency = zip(*top_bigrams)

    # Create a bar plot for bigrams
    plt.figure(figsize=(10, 6))
    plt.bar([" ".join(bg) for bg in bigram], frequency)
    plt.title(f"Top Bigrams in {trope_name}")
    plt.xlabel("Bigrams")
    plt.ylabel("Frequency")
    plt.xticks(rotation=45)

    # Adjust the margins to prevent text from getting cut off
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    # Save the bar chart to a file in the output directory
    image_file_path = os.path.join(output_directory, f"{trope_name}_bigram_bar_chart.png")
    plt.savefig(image_file_path)
    plt.close()  # Close the plot to release resources

# All bigram bar charts have been saved to the output_directory

Again, it's not very meaningful to have just 1 occurence of a bigram. Hence, we're setting a minimum limit of 2.

In [17]:
# Create a directory to save the bar chart images
output_directory = "bar_charts_trigrams"
os.makedirs(output_directory, exist_ok=True)

# Loop through each trope and perform EDA on bigrams
for trope_name in trope_names:
    # Filter the data for the current trope
    trope_data = ten_tropes_df[ten_tropes_df["Trope Name"] == trope_name]

    # Tokenize the lemmatized descriptions and remove common stopwords and custom stopwords
    all_words = " ".join(trope_data["text_lemma"])
    all_words = word_tokenize(all_words)
    
    # Remove common stopwords
    filtered_words = [word for word in all_words if word.lower() not in nltk_stopwords]
    
    # Remove custom stopwords
    filtered_words = [word for word in filtered_words if word.lower() not in custom_stopwords.get(trope_name, [])]

    # Create bigrams
    bigrams = list(ngrams(filtered_words, 2))

    # Count bigram frequencies
    bigram_freq = Counter(bigrams)

    # Set a threshold for bigram frequency (e.g., 2 or more occurrences)
    filtered_bigrams = {bg: freq for bg, freq in bigram_freq.items() if freq > 2}

    if filtered_bigrams:
        # Visualize the top occurring bigrams
        top_bigrams = Counter(filtered_bigrams).most_common(10)
        bigram, frequency = zip(*top_bigrams)

        # Create a bar plot for bigrams
        plt.figure(figsize=(10, 6))
        plt.bar([" ".join(bg) for bg in bigram], frequency)
        plt.title(f"Top Bigrams in {trope_name}")
        plt.xlabel("Bigrams")
        plt.ylabel("Frequency")
        plt.xticks(rotation=45)

        # Adjust the margins to prevent text from getting cut off
        plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    # Save the bar chart to a file in the output directory
    image_file_path = os.path.join(output_directory, f"{trope_name}_2_minimum_bigram_bar_chart.png")
    plt.savefig(image_file_path)
    plt.close()  # Close the plot to release resources

We might have to do a trigram.

In [18]:
# Loop through each trope and perform EDA on trigrams
for trope_name in trope_names:
    # Filter the data for the current trope
    trope_data = ten_tropes_df[ten_tropes_df["Trope Name"] == trope_name]

    # Tokenize the lemmatized descriptions and remove common stopwords and custom stopwords
    all_words = " ".join(trope_data["text_lemma"])
    all_words = word_tokenize(all_words)
    
    # Remove common stopwords
    filtered_words = [word for word in all_words if word.lower() not in nltk_stopwords]
    
    # Remove custom stopwords
    filtered_words = [word for word in filtered_words if word.lower() not in custom_stopwords.get(trope_name, [])]

    # Create trigrams
    trigrams = list(ngrams(filtered_words, 3))

    # Count trigram frequencies
    trigram_freq = Counter(trigrams)

    # Set a threshold for trigram frequency (e.g., 2 or more occurrences)
    filtered_trigrams = {tg: freq for tg, freq in trigram_freq.items() if freq > 2}

    if filtered_trigrams:
        # Visualize the top occurring trigrams
        top_trigrams = Counter(filtered_trigrams).most_common(10)
        trigram, frequency = zip(*top_trigrams)

        # Create a bar plot for trigrams
        plt.figure(figsize=(10, 6))
        plt.bar([" ".join(tg) for tg in trigram], frequency)
        plt.title(f"Top Trigrams in {trope_name}")
        plt.xlabel("Trigrams")
        plt.ylabel("Frequency")
        plt.xticks(rotation=45)

        # Adjust the margins to prevent text from getting cut off
        plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

        # Create a subdirectory within the main output directory for trigrams
        trigram_output_directory = os.path.join(output_directory, "trigrams")
        os.makedirs(trigram_output_directory, exist_ok=True)
        
        # Save the bar chart to a file in the output directory
        image_file_path = os.path.join(output_directory, f"{trope_name}_2_minimum_trigram_bar_chart.png")
        plt.savefig(image_file_path)
        plt.close()  # Close the plot to release resources

In [19]:
X, y = make_multilabel_classification(n_classes=3, random_state=0)
clf = MultiOutputClassifier(LogisticRegression()).fit(X, y)
clf.predict(X[-2:])

array([[1, 1, 1],
       [1, 0, 1]])

In [21]:
# Define a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Adjust the number of features as needed

# Create a list to store features
X = []

# Loop through each trope and perform EDA
for trope_name in trope_names:
    # Filter the data for the current trope
    trope_data = ten_tropes_df[ten_tropes_df["Trope Name"] == trope_name]

    # Extract the tokenized, lemmatized, and cleaned text data (features)
    all_words = " ".join(trope_data["text_lemma"])
    all_words = word_tokenize(all_words)
    filtered_words = [word for word in all_words if word.lower() not in nltk_stopwords]
    filtered_words = [word for word in filtered_words if word.lower() not in custom_stopwords.get(trope_name, [])]

    # Add features to X
    X.append(" ".join(filtered_words))  # Convert the list of words back to a single string

# Vectorize the features using TF-IDF
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Perform clustering (e.g., K-means clustering) and specify n_init
n_clusters = 5  # Adjust the number of clusters as needed
n_init_value = 10  # Set the value explicitly to suppress the warning
kmeans = KMeans(n_clusters=n_clusters, n_init=n_init_value, random_state=0)
kmeans.fit(X_tfidf)

# Assign cluster labels to each trope
cluster_labels = kmeans.labels_

# Print out the cluster labels for each trope
for trope_name, label in zip(trope_names, cluster_labels):
    print(f"Trope: {trope_name}, Cluster Label: {label}")

# Now you have cluster labels for your tropes

Trope: Imagine Spot, Cluster Label: 3
Trope: Couch Gag, Cluster Label: 0
Trope: Catchphrase, Cluster Label: 4
Trope: Comically Missing the Point, Cluster Label: 1
Trope: Running Gag, Cluster Label: 0
Trope: Disproportionate Retribution, Cluster Label: 1
Trope: Establishing Character Moment, Cluster Label: 2
Trope: Karma Houdini, Cluster Label: 1
Trope: Characterization Marches On, Cluster Label: 2
Trope: Special Guest, Cluster Label: 4


In [28]:
# Assuming cluster_labels is a list of cluster labels for each trope
# For example, cluster_labels = [3, 0, 4, 1, 0, 1, 2, 1, 2, 4]

# Create binary labels based on cluster labels
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_combined = mlb.fit_transform([f"Cluster{cluster_label}" for cluster_label in cluster_labels])

# Now y_combined is a binary label matrix that can be used with make_multilabel_classification

# Print the transformed binary labels
y_combined

array([[0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [29]:
# Print the inverse transformation to check the mapping
original_labels = mlb.inverse_transform(y_combined)
original_labels

[('3', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u'),
 ('0', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u'),
 ('4', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u'),
 ('1', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u'),
 ('0', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u'),
 ('1', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u'),
 ('2', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u'),
 ('1', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u'),
 ('2', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u'),
 ('4', 'C', '[', ']', 'e', 'l', 'r', 's', 't', 'u')]